In [1]:
import tensorflow as tf
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [12]:
# df = pd.read_csv('AirPassengers.csv')
# df.head()



zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

13568290/13568290 [==============================] - 43s 3us/step


In [13]:
df = pd.read_csv(csv_path)
df.head()

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


In [14]:
plt.plot(df['Date Time'], df['T (degC)'])

craete dataset

In [ ]:
def df_to_X_y(df, window_size=5):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [[a] for a in df_as_np[i:i+window_size]]
    X.append(row)
    label = df_as_np[i+window_size]
    y.append(label)
  return np.array(X), np.array(y)

In [6]:
WINDOW_SIZE = 5
X1, y1 = df_to_X_y(df['T (degC)'], WINDOW_SIZE)
X1.shape, y1.shape

((139, 5, 1), (139,))

In [7]:
X_train1, y_train1 = X1[:100], y1[:100]
X_val1, y_val1 = X1[100:120], y1[100:120]
X_test1, y_test1 = X1[120:], y1[120:]
X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape

((100, 5, 1), (100,), (20, 5, 1), (20,), (19, 5, 1), (19,))

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

model1 = Sequential()
model1.add(InputLayer((5, 1)))
model1.add(LSTM(64))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear'))

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dense (Dense)               (None, 8)                 520       
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 17,425
Trainable params: 17,425
Non-trainable params: 0
_________________________________________________________________


In [9]:

model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])

In [10]:
model1.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=10)

Epoch 1/10
4/4 [==============================] - 5s 139ms/step - loss: 61182.9688 - root_mean_squared_error: 247.3519 - val_loss: 142563.1719 - val_root_mean_squared_error: 377.5754
Epoch 2/10
4/4 [==============================] - 0s 14ms/step - loss: 61169.4805 - root_mean_squared_error: 247.3246 - val_loss: 142541.6562 - val_root_mean_squared_error: 377.5469
Epoch 3/10
4/4 [==============================] - 0s 14ms/step - loss: 61156.7500 - root_mean_squared_error: 247.2989 - val_loss: 142521.5938 - val_root_mean_squared_error: 377.5203
Epoch 4/10
4/4 [==============================] - 0s 14ms/step - loss: 61144.5195 - root_mean_squared_error: 247.2742 - val_loss: 142501.6719 - val_root_mean_squared_error: 377.4939
Epoch 5/10
4/4 [==============================] - 0s 13ms/step - loss: 61132.1758 - root_mean_squared_error: 247.2492 - val_loss: 142481.1562 - val_root_mean_squared_error: 377.4668
Epoch 6/10
4/4 [==============================] - 0s 12ms/step - loss: 61119.4102 - root_

In [11]:
train_predictions = model1.predict(X_train1).flatten()
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y_train1})
train_results

4/4 [==============================] - 0s 3ms/step


,Train Predictions,Actuals
0,0.687990,135
1,0.690444,148
2,0.693638,148
3,0.694373,136
4,0.692854,119
...,...,...
95,0.738926,355
96,0.739803,422
97,0.748330,465
98,0.753075,467
